In [3]:
!pip install -U transformers
!pip install -U datasets
!pip install tensorboard
!pip install sentencepiece
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 89.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.2
    Uninstalling transformers-4.52.2:
      Successfully uninstalled transformers-4.52.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is i

In [1]:
!wget "https://www.dropbox.com/scl/fi/525gv6tmdi3n32mipo6mr/input.zip?rlkey=5jdsxahphk2ped5wxbxnv0n4y&dl=1" -O input.zip


--2025-06-03 07:53:12--  https://www.dropbox.com/scl/fi/525gv6tmdi3n32mipo6mr/input.zip?rlkey=5jdsxahphk2ped5wxbxnv0n4y&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0fe800a04981d7bddd16a759d0.dl.dropboxusercontent.com/cd/0/inline/Cq5HroK549TJ5p5yyNnriUC_PvYdhU-Kft6_WRNdzH66E2TjDdJuWo3q5RMfIkOW3N6a09k4SKdxX9ouh-rBD7dg2I6ZIjpwzb1lMdJKf6LldxLT2tCuwQfkoHaZFWC9zj1UWV9BhuDrxFzCmeoZLWiM/file?dl=1# [following]
--2025-06-03 07:53:13--  https://uc0fe800a04981d7bddd16a759d0.dl.dropboxusercontent.com/cd/0/inline/Cq5HroK549TJ5p5yyNnriUC_PvYdhU-Kft6_WRNdzH66E2TjDdJuWo3q5RMfIkOW3N6a09k4SKdxX9ouh-rBD7dg2I6ZIjpwzb1lMdJKf6LldxLT2tCuwQfkoHaZFWC9zj1UWV9BhuDrxFzCmeoZLWiM/file?dl=1
Resolving uc0fe800a04981d7bddd16a759d0.dl.dropboxusercontent.com (uc0fe800a04981d7bddd16a759d0.dl.dropboxusercontent.com)... 162.125.

In [2]:
!unzip input.zip


Archive:  input.zip
   creating: input/
  inflating: input/train.csv         
  inflating: input/valid.csv         


In [3]:
import torch

from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)
from datasets import load_dataset

In [28]:
MODEL = 't5-small'
BATCH_SIZE = 48
NUM_PROCS = 16
EPOCHS = 2
OUT_DIR = 'results_t5small'
MAX_LENGTH = 256 # Maximum context length to consider while preparing dataset.

In [7]:
dataset_train = load_dataset(
    'csv',
    data_files='input/train.csv',
    split='train'
)
dataset_valid = load_dataset(
    'csv',
    data_files='input/valid.csv',
    split='train'
)

In [8]:
print(dataset_train)
print(dataset_valid)

Dataset({
    features: ['Id', 'Title', 'Body', 'Tags', 'CreationDate', 'Y'],
    num_rows: 45000
})
Dataset({
    features: ['Id', 'Title', 'Body', 'Tags', 'CreationDate', 'Y'],
    num_rows: 15000
})


In [10]:
print(dataset_train[0])


{'Id': 34552656, 'Title': 'Java: Repeat Task Every Random Seconds', 'Body': '<p>I\'m already familiar with repeating tasks every n seconds by using Java.util.Timer and Java.util.TimerTask. But lets say I want to print "Hello World" to the console every random seconds from 1-5. Unfortunately I\'m in a bit of a rush and don\'t have any code to show so far. Any help would be apriciated.  </p>\n', 'Tags': '<java><repeat>', 'CreationDate': '2016-01-01 00:21:59', 'Y': 'LQ_CLOSE'}


In [11]:
tokenizer = T5Tokenizer.from_pretrained(MODEL)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [12]:
# Function to convert text data into model inputs and targets
def preprocess_function(examples):
    inputs = [f"assign tag: {title} {body}" for (title, body) in zip(examples['Title'], examples['Body'])]
    model_inputs = tokenizer(
        inputs,
        max_length=MAX_LENGTH,
        truncation=True,
        padding='max_length'
    )

    # Set up the tokenizer for targets
    cleaned_tag = [' '.join(''.join(tag.split('<')).split('>')[:-1]) for tag in examples['Tags']]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            cleaned_tag,
            max_length=MAX_LENGTH,
            truncation=True,
            padding='max_length'
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the function to the whole dataset
tokenized_train = dataset_train.map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS
)
tokenized_valid = dataset_valid.map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS
)

Map (num_proc=16):   0%|          | 0/45000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

Map (num_proc=16):   0%|          | 0/15000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

In [13]:
print(tokenized_train[0])

{'Id': 34552656, 'Title': 'Java: Repeat Task Every Random Seconds', 'Body': '<p>I\'m already familiar with repeating tasks every n seconds by using Java.util.Timer and Java.util.TimerTask. But lets say I want to print "Hello World" to the console every random seconds from 1-5. Unfortunately I\'m in a bit of a rush and don\'t have any code to show so far. Any help would be apriciated.  </p>\n', 'Tags': '<java><repeat>', 'CreationDate': '2016-01-01 00:21:59', 'Y': 'LQ_CLOSE', 'input_ids': [12317, 7860, 10, 10318, 10, 20469, 16107, 2181, 25942, 5212, 7, 3, 2, 102, 3155, 196, 31, 51, 641, 3324, 28, 6103, 53, 4145, 334, 3, 29, 3978, 57, 338, 10318, 5, 13780, 5, 13368, 52, 11, 10318, 5, 13780, 5, 13368, 52, 382, 9, 7, 157, 5, 299, 8857, 497, 27, 241, 12, 2281, 96, 566, 7126, 1150, 121, 12, 8, 8990, 334, 6504, 3978, 45, 209, 4525, 5, 4877, 27, 31, 51, 16, 3, 9, 720, 13, 3, 9, 10505, 11, 278, 31, 17, 43, 136, 1081, 12, 504, 78, 623, 5, 2372, 199, 133, 36, 3, 9, 2246, 4915, 1054, 5, 3, 2, 87, 1

In [14]:
model = T5ForConditionalGeneration.from_pretrained(MODEL)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

60,506,624 total parameters.
60,506,624 training parameters.


In [19]:
!pip install --upgrade transformers

In [29]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=OUT_DIR,
    logging_steps=10,
    eval_steps=500,  # Evaluation every 500 steps
    eval_strategy="steps",  # ✅ MATCHES save_strategy
    save_strategy="steps",        # Optional, since this is the default when using `eval_steps`
    load_best_model_at_end=True,
    save_total_limit=5,
    report_to='tensorboard',
    learning_rate=0.0001,
    fp16=True,
    dataloader_num_workers=4
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
)

history = trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
500,0.089700,0.080675
1000,0.074200,0.067817
1500,0.070400,0.064146


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'd

In [30]:
tokenizer.save_pretrained(OUT_DIR)

('results_t5small/tokenizer_config.json',
 'results_t5small/special_tokens_map.json',
 'results_t5small/spiece.model',
 'results_t5small/added_tokens.json')

In [31]:
!wget "https://www.dropbox.com/scl/fi/9brsjizymq5zvqi7hff09/inference_data.zip?rlkey=ukmdy5egmdld80r5hhmsja78v&dl=1" -O inference_data.zip


--2025-06-03 08:29:41--  https://www.dropbox.com/scl/fi/9brsjizymq5zvqi7hff09/inference_data.zip?rlkey=ukmdy5egmdld80r5hhmsja78v&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc81d88bcdb8c250452e8f6ece4c.dl.dropboxusercontent.com/cd/0/inline/Cq74Moxb81-alyab96YFlUfozhy7UsnJJNBabrQFmCk4S9iBS82sD9vY2AUdnRxbkg8AL5B1MDKBkzVr99zUowGj4wHOBTmKTkfWuyNZ01fNOent_f5nGm7Yot2e1z47nLry_z0b1RAvdmWmkWQsYweo/file?dl=1# [following]
--2025-06-03 08:29:42--  https://uc81d88bcdb8c250452e8f6ece4c.dl.dropboxusercontent.com/cd/0/inline/Cq74Moxb81-alyab96YFlUfozhy7UsnJJNBabrQFmCk4S9iBS82sD9vY2AUdnRxbkg8AL5B1MDKBkzVr99zUowGj4wHOBTmKTkfWuyNZ01fNOent_f5nGm7Yot2e1z47nLry_z0b1RAvdmWmkWQsYweo/file?dl=1
Resolving uc81d88bcdb8c250452e8f6ece4c.dl.dropboxusercontent.com (uc81d88bcdb8c250452e8f6ece4c.dl.dropboxusercontent.com)...

In [32]:
!unzip inference_data.zip


Archive:  inference_data.zip
   creating: inference_data/
  inflating: inference_data/text_1.txt  
  inflating: inference_data/text_2.txt  
  inflating: inference_data/text_3.txt  
  inflating: inference_data/text_4.txt  
  inflating: inference_data/text_5.txt  
  inflating: inference_data/text_6.txt  


In [33]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

import os

In [40]:
import os

# List all contents of your model output directory
print(os.listdir("results_t5small"))

['checkpoint-1876', 'special_tokens_map.json', 'checkpoint-1000', 'events.out.tfevents.1748937805.48653ec3094a.1532.1', 'checkpoint-1500', 'tokenizer_config.json', 'spiece.model', 'checkpoint-500', 'added_tokens.json', 'events.out.tfevents.1748937737.48653ec3094a.1532.0', 'events.out.tfevents.1748937827.48653ec3094a.1532.2']


In [41]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_path = 'results_t5small/checkpoint-1876/'  # Local checkpoint folder

model = T5ForConditionalGeneration.from_pretrained(model_path, local_files_only=True)
tokenizer = T5Tokenizer.from_pretrained('results_t5small', local_files_only=True)

In [42]:
def do_correction(text, model, tokenizer):
    input_text = f"assign tag: {text}"
    inputs = tokenizer.encode(
        input_text,
        return_tensors='pt',
        max_length=256,
        padding='max_length',
        truncation=True
    )

    # Get correct sentence ids.
    corrected_ids = model.generate(
        inputs,
        max_length=256,
        num_beams=5, # `num_beams=1` indicated temperature sampling.
        early_stopping=True
    )

    # Decode.
    corrected_sentence = tokenizer.decode(
        corrected_ids[0],
        skip_special_tokens=True
    )
    return corrected_sentence

In [43]:
for file in os.listdir('inference_data/'):
    f = open(f"inference_data/{file}", 'r')
    sentence = f.read()
    corrected_sentence = do_correction(sentence, model, tokenizer)
    print(f"QUERY: {sentence}\nTAGS: {corrected_sentence}")
    print('-'*100)

QUERY: How do I change the grid size with Matplotlib?
This is what I want:

properly spaced grid

This is what I currently have:

my grid

This is my code so far:

l = [-1, 1, -10, 10]
plt.axis(l)
plt.grid()

TAGS: matplotlib
----------------------------------------------------------------------------------------------------
QUERY: Repeat Task Every Random Seconds <p>I\'m already familiar with repeating tasks every n seconds by using Java.util.Timer and Java.util.TimerTask. But lets say I want to print \"Hello World\" to the console every random seconds from 1-5. Unfortunately I\'m in a bit of a rush and don\'t have any code to show so far. Any help would be apriciated

TAGS: java
----------------------------------------------------------------------------------------------------
QUERY: C++: `error: expected ',' or '...' before string constant` before `__FILE__` in `fancy_abort (__FILE__, __LINE__, __FUNCTION__)` when compiling gcc
This is a continuation of trying to compile a license-